In [1]:
import os
import requests as req
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr
from IPython.display import Markdown, display, update_display

<h1>BUILDING UP A CHATBOT</h1>

In [2]:
system_message = "you are an helpful assistant"


In [3]:
# system_message+="you are handling the business of green roots that for the hairall solution and strenghting\
# and moisture the hair, which make them grow fast"

In [4]:
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise RuntimeError("Missing ANTHROPIC_API_KEY in .env")

from anthropic import Anthropic
client = Anthropic(api_key=api_key)
Model = "claude-sonnet-4-20250514"

In [5]:
def chat(message, history):
    
    claude_history = []
    for m in history:
        if m["role"] in ["user", "assistant"]:  
            claude_history.append({
                "role": m["role"],
                "content": [{"type": "text", "text": m["content"]}]
            })

   
    claude_history.append({
        "role": "user",
        "content": [{"type": "text", "text": message}]
    })

    
    response = client.messages.create(
        model=Model,
        max_tokens=500,
        system=system_message,  
        messages=claude_history
    )

    return response.content[0].text


In [6]:
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://0a0a5522c433958398.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
def chat_ollama(message, history):
    messages = (
        [{"role": "system", "content": system_message}]   # wrap system in a list
        + history                                         # history is already a list of {role, content}
        + [{"role": "user", "content": message}]          # wrap current user message in a list
    )

    resp = req.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3",
            "messages": messages,
            "stream": False
        }
    )
    data = resp.json()
    return data["message"]["content"]


In [9]:
gr.ChatInterface(fn=chat_ollama, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
